In [1]:
#/pip install opencv-python
#pip install mediapipe
#pip install tensorflow

In [2]:
import cv2

cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        out.write(frame)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
out.release()
cv2.destroyAllWindows()


2024-07-26 13:51:21.291 Python[3319:73441] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


pose estimation model

In [3]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.pose_landmarks:
             cv2.imshow('frame',frame)
            # do something with the landmarks
             
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1721982094.044814   73441 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1721982094.150377   73840 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1721982094.157475   73847 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Applications/ms/.venv/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Building the Personal Trainer

In [4]:
exercises = {
    'Squats': {
        'nose': (0.4, 0.6),
        'left_shoulder': (0.2, 0.4),
        'right_shoulder': (0.6, 0.8),
        'left_hip': (0.2, 0.4),
        'right_hip': (0.6, 0.8),
        'left_knee': (0.4, 0.6),
        'right_knee': (0.4, 0.6),
        'left_ankle': (0.2, 0.4),
        'right_ankle': (0.6, 0.8)
    },
    'Pushups': {
        'nose': (0.4, 0.6),
        'left_shoulder': (0.2, 0.4),
        'right_shoulder': (0.6, 0.8),
        'left_elbow': (0.2, 0.4),
        'right_elbow': (0.6, 0.8),
        'left_wrist': (0.2, 0.4),
        'right_wrist': (0.6, 0.8)
    }
}

In the above code, we define the correct pose for two exercises, Squats and Pushups. For each exercise, we define the acceptable range of values for each landmark returned by the pose estimation model.

Next, we need to define a function to check if the person is performing the exercise correctly. We will check if the landmarks returned by the pose estimation model fall within the acceptable range of values defined for the exercise.

In [5]:
def check_exercise(landmarks, exercise):
    for landmark, range in exercise.items():
        x = landmarks[landmark].x
        y = landmarks[landmark].y
        if x < range[0] or x > range[1] or y < range[0] or y > range[1]:
            return False
    return True

In the above code, we define a function to check if the landmarks fall within the acceptable range of values for the exercise. The function returns True if the landmarks fall within the acceptable range of values, otherwise it returns False.

Now that we can check if the person is performing the exercise correctly, we can provide feedback in real-time. We can draw lines and circles on the video to indicate the correct posture for the exercise. We can also display text on the video to provide feedback on the person’s performance.



In [6]:
def draw_pose(frame, landmarks, exercise):
    for landmark, _ in exercise.items():
        x = int(landmarks[landmark].x * frame.shape[1])
        y = int(landmarks[landmark].y * frame.shape[0])
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
    cv2.putText(frame, "Correct posture", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.pose_landmarks:
            landmarks = {}
            for i, landmark in enumerate(results.pose_landmarks.landmark):
                landmarks[f'landmark_{i}'] = landmark
            if check_exercise(landmarks, exercises['Squats']):
                draw_pose(frame, landmarks, exercises['Squats'])
            elif check_exercise(landmarks, exercises['Pushups']):
                draw_pose(frame, landmarks, exercises['Pushups'])
            else:
                cv2.putText(frame, "Incorrect posture", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

KeyError: 'nose'

In the above code, we capture frames from the webcam and pass them to the pose estimation model.

We then check if the person is performing Squats or Pushups correctly using the check_exercise() function. If the person is performing the exercise correctly, we draw the correct pose on the video using the draw_pose() function.

If the person is not performing the exercise correctly, we display a message on the video indicating incorrect posture.

